# Megatron ForumQA

## CD

定位到工作目录，根据具体情况决定哦，不一定是下面的命令

In [1]:
%cd ..

/home/Public/Megatron-LM


## 环境准备

准备运行这个笔记本的 Jupyter kernel(**如果已经准备就绪，不要重复执行！**)：


1. 配置一个 Conda 环境作为 Jupyter Kernel

In [ ]:
%conda env update -f environments/environment-ipy.yml

安装完毕后，为该 Notebook 选择这个 Kernel (名为`Megatron_LM-ipy`)

2. 在Kernel所在 Conda 环境中安装 Apex

需要通过 pip 从 github 下载源代码安装：

In [ ]:
%pip install -v -r requirements/apex.txt

## 下载 Checkpoints

文件比较大，根据实际情况选择下载，**不要重复下载**

In [2]:
import os

S3_BUCKET = 'huamei'
CKPTS_DIR = 'checkpoints/345m-mildil'

S3_CKPTS_DIR = 's3://' + os.path.join(S3_BUCKET, CKPTS_DIR)

In [3]:
%%time

# 复制 latest_checkpointed_iteration.txt
!aws s3 cp \
    {S3_CKPTS_DIR} \
    {CKPTS_DIR} \
    --recursive \
    --exclude "*" \
    --include "latest_checkpointed_iteration.txt"

# 下载后读取最新的 checkpoint iter 名称
iter_step = open(f'{CKPTS_DIR}/latest_checkpointed_iteration.txt').read().strip()
ckpt_dir = f'iter_{iter_step}'

print(f'checkpoint {ckpt_dir}')

s3_ckpt_dir = os.path.join(S3_CKPTS_DIR, ckpt_dir)
local_ckpt_dir = os.path.join(CKPTS_DIR, ckpt_dir)

print(f'sync: {s3_ckpt_dir} -> {local_ckpt_dir}')
    
# 同步最新的 Checkpiont
!aws s3 sync \
    s3://huamei/hmgpt2-checkpoints/345m-hmwebmix-bpe-v2/iter_0230000 \
    ./checkpoints/345m-hmwebmix-bpe-v2/iter_0230000


checkpoint iter_230000
sync: s3://huamei/checkpoints/345m-hmwebmix-bpe-v2/iter_230000 -> checkpoints/345m-hmwebmix-bpe-v2/iter_230000
CPU times: user 27.4 ms, sys: 21.1 ms, total: 48.5 ms
Wall time: 2.26 s


## Environment Variables

- 用哪个/些 GPU?

In [3]:
%env CUDA_VISIBLE_DEVICES 0

env: CUDA_VISIBLE_DEVICES=0


## Importings

In [2]:
import copy
import csv
import json
import math
import os
import random
import sys
import time
from contextlib import closing
from itertools import chain, compress
from functools import partial
from multiprocessing import Pool
from types import SimpleNamespace

import numpy as np
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm, trange

import mpu
from data_utils.tokenization import SentencePieceTokenizer, make_tokenizer
from pretrain_gpt2 import get_masks_and_position_ids
from predict_gpt2 import initialize_distributed, prepare_tokenizer, set_random_seed, setup_model, get_token_stream

## Args

In [3]:
MODELS_HPARAMS_DICT = {
    '117m': dict(
        num_layers=12,
        hidden_size=768,
        num_attention_heads=12,
        max_position_embeddings=1024,
    ),
    '345m': dict(
        num_layers=24,
        hidden_size=1024,
        num_attention_heads=16,
        max_position_embeddings=1024,
    )
}

In [4]:
PARAMETERS = '117m'

In [5]:
args = SimpleNamespace(
    # Model arguments
    vocab_size=None,
    make_vocab_size_divisible_by=128,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    # Train/valid/test data arguments.
    seq_length=1024,
    model_parallel_size=1,
    tokenizer_model_type='bert-large-uncased',
#     tokenizer_type='GPT2BPETokenizer_CN',
    tokenizer_type='SentencePieceTokenizer',
    tokenizer_path="./data/spm/gpt2_huamei_corpus_bpe_32k_v2.model",
    cache_dir=None,
    # Training arguments.
    load='./checkpoints/xinliqa.117m/',
    seed=1234,
    checkpoint_activations=None,
    checkpoint_num_layers=1,
    finetune=None,
    no_load_optim=None,
    no_load_rng=None,
    resume_dataloader=None,
    fp16=True,
    hysteresis=2,
    loss_scale=None,
    loss_scale_window=1000,
    min_scale=1,
    distributed_backend='nccl',
    DDP_impl='local',
    local_rank=None,
    reset_position_ids=None,
    reset_attention_mask=None,
    eod_mask_loss=None, 
    # Text generate arguments.
    recompute=None,
    greedy=False,
    top_p=0.0,
    top_k=0,
    temperature=1.0,
    out_seq_length=1024,
)

for k, v in MODELS_HPARAMS_DICT[PARAMETERS].items():
    setattr(args, k, v)

In [6]:
with open(os.path.join(args.load, 'latest_checkpointed_iteration.txt')) as fp:
    latest_checkpointed_iteration = int(fp.read().strip())
print('latest_checkpointed_iteration:', latest_checkpointed_iteration)

latest_checkpointed_iteration: 180000


In [7]:
args.cuda = torch.cuda.is_available()
args.rank = int(os.getenv('RANK', '0'))
args.world_size = int(os.getenv("WORLD_SIZE", '1'))

if os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'):
    # We are using (OpenMPI) mpirun for launching distributed data parallel processes
    local_rank = int(os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'))
    local_size = int(os.getenv('OMPI_COMM_WORLD_LOCAL_SIZE'))

    # Possibly running with Slurm
    num_nodes = int(os.getenv('SLURM_JOB_NUM_NODES', '1'))
    nodeid = int(os.getenv('SLURM_NODEID', '0'))

    args.local_rank = local_rank
    args.rank = nodeid*local_size + local_rank
    args.world_size = num_nodes*local_size

args.model_parallel_size = min(args.model_parallel_size, args.world_size)
if args.rank == 0:
    print('using world size: {} and model-parallel size: {} '.format(
        args.world_size, args.model_parallel_size))

args.dynamic_loss_scale = False
if args.loss_scale is None:
    args.dynamic_loss_scale = True
    if args.rank == 0:
        print(' > using dynamic loss scaling')

# The args fp32_* or fp16_* meant to be active when the
# args fp16 is set. So the default behavior should all
# be false.
if not args.fp16:
    args.fp32_embedding = False
    args.fp32_tokentypes = False
    args.fp32_layernorm = False


using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling


## Init

### 初始化函数/全局变量

In [8]:
tokenizer = None
model = None

def initialize():
    global model, tokenizer

    # Disable CuDNN.
    torch.backends.cudnn.enabled = False

    # Pytorch distributed.
    initialize_distributed(args)

    # Random seeds for reproducability.
    set_random_seed(args.seed)

    # get the tokenizer
    tokenizer = prepare_tokenizer(args)

    # Model, optimizer, and learning rate.
    model = setup_model(args)

    args.device = torch.cuda.current_device()

    # setting default batch size to 1
    args.batch_size = 1

    assert mpu.get_model_parallel_rank() == 0

### 主进程初始化

In [9]:
%%time

initialize()


> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
prepare tokenizer done
building GPT2 model ...
 > number of parameters on model parallel rank 0: 110516736
global rank 0 is loading checkpoint ./checkpoints/xinliqa.117m/iter_0180000/mp_rank_00/model_optim_rng.pt
  successfully loaded ./checkpoints/xinliqa.117m/iter_0180000/mp_rank_00/model_optim_rng.pt
CPU times: user 5.19 s, sys: 1.62 s, total: 6.81 s
Wall time: 6.82 s


## 预测用的函数

In [10]:
def infer_tokens_generative(context_tokens, model, tokenizer):
    context_length = len(context_tokens)
    token_stream = get_token_stream(model, [context_tokens], tokenizer, args)   
    for i, (output_tokens, _) in enumerate(token_stream):
        if context_length + i >= args.seq_length:
            break
        ids = output_tokens.cpu().numpy().tolist()[0]
        yield ids[-1]


def infer_text_generative(contex_text, model, tokenizer):
    contex_text = contex_text.strip()
    context_tokens = tokenizer.EncodeAsIds(contex_text).tokenization
    context_length = len(context_tokens)

    token_stream = get_token_stream(model, [context_tokens], tokenizer, args)
    
    for i, (output_tokens, _) in enumerate(token_stream):
        if context_length + i >= args.seq_length:
            break
        ids = output_tokens.cpu().numpy().tolist()[0]
        s = tokenizer.DecodeIds([ids[-1]])
        yield s


def question_to_ids(data, end_with_bos=True):
    result = []
    # 标题, 问题
    for k in ('title', 'text'):
        s = data[k].strip()
        ids = tokenizer.EncodeAsIds(s.strip()).tokenization
        result.extend(ids)
        result.append(tokenizer.TokenToId('<sep>'))
    tags = data.get('tags')
    if tags:
        tag = random.choice(tags)
        ids = tokenizer.EncodeAsIds(tag.strip()).tokenization
        result.extend(ids)
    result.append(tokenizer.TokenToId('<sep>'))
    # 问题/回答 的分隔符，保证连续两个 `<sep>`
    result.append(tokenizer.TokenToId('<sep>'))
    # 回答 的开头 `<bos>`
    if end_with_bos:
        result.append(tokenizer.TokenToId('<bos>'))
    #
    return result


## 验证是否可运行

In [11]:
question_list = [
    {
        "title": "22岁，做直销",
        "text": "父母都很支持，可是我性格内向，而且不喜欢说话，圈内有很多成功了的大神，我没自信能不能成功，直销靠谱不",
        "tags": ["职业", "职业管理", "工作压力"]
    },
    {
        "title": "真理都是相通的?",
        "text": "从洞穴出来看见理性之光的形而上人开始拼凑真理的碎片，到达二点零的人拼凑起来啦一部分碎片形成自己一套看世界的体系。\n这时候的人都会用自己的一套概念体系去描述他研究得那一部分真理，可是你会发现他们说得都是一回事——真理都是相通的只是表述不同罢啦\n人性是人文科学的研究对象，物性是自然科学的研究对象，前者合乎情理（符合哲理），后者合乎逻辑（符合形式逻辑那一套），宗教的研究对象是信仰，哲学是哲学家，美学是艺术家。\n真理的高山一直在那里，我们已经探索啦很多很多，我们学习学科历史是要把山头探索清楚然后摸清楚魔方块的套路，最后构建自己的魔方块",
        "tags": ["科普", "书籍", "热点话题", "心理咨询"]
    },
]

In [12]:
# args.recompute=True
# args.top_p=0.0
# args.top_k=0
# args.temperature=1.0

n_gen = 3
max_try = 3

for question in question_list:
#     print(question['title'])
#     print(question['text'])
#     input_ids = question_to_ids(question)
#     print(tokenizer.DecodeIds(input_ids))
#     print()

    for i in range(n_gen):
#         args.temperature=random.gauss(0.95, 0.05)
        print(f'Answer {i+1}: ', end='')
        s = ''
        for _ in range(max_try):
            if s:
                break
            input_ids = question_to_ids(question)
            for j, id_ in enumerate(infer_tokens_generative(input_ids, model, tokenizer)):
                if j == 0:
#                     if id_ != tokenizer.TokenToId('<bos>'):
#                         break
                    if id_ == tokenizer.TokenToId('<eos>'):
                        break
                s = tokenizer.DecodeIds([id_])
                print(s, end='')
        print()
    print()
    print('=' * 100)
    print()
    

Answer 1: ,我都会嘲笑我想,见识的很二三任咨询师如何都只会做着变成说过话,真到明了,但我正用穿着女性脱精液来毁容取女,我感觉很反感。总是一个家店女同事衣着穿,三观逐步起来,我发现,脸色差不多,随着年龄的增长,脾气比较萌,也越来越像她吹毛求疵,我也不是邪,最重要的是小孩做不到,我知道这样让自己疑神病一样的,我家人说是妹妹让我不要把孩子气剪的,于是我拼命的抓,不仅不听话还天天说我在那里等着,还动不动说别的小孩,我看着不舒服,我都快气疯了,我要打我妈了。对了,说我是青春期的女生,那我再怎么长身体呢,应该和她一样傻傻的接受治疗吗?作为子女,我该怎么做? <eos>
Answer 2: 事社团,我却收益,我不知道初中的人资视乐都说不出会就长大些太少,比较像小的学生、大方选采购。从开学到现在不了解,不知道为什么会不知道为什么,每次你我来提问到现在,我常常问兵老神从其正确答,他说结婚哟我以为是,蜜蜂蜜,做普工的像,给他女儿画笔快乐。可是,后妈姐姐弟弟态度很亢奋,严重影响了学习考大学等,出来工作几年,也为了收益累积,拒绝回家。我很少在公开场合发言,大学期间偶尔亲亲昵撒娇,也被拉黑,以致那段时间被当众演讲的面试管说我是傻13名,从此我就有了对自己不满意的疏远情绪,为了避免来往还当面冷场。后来出院后,我主动热情的接待了她,说我是脑子有问题,然后她问了一句话让我闭嘴。我其实活得很累,有一点胆小,害羞,缺少自信心,我很怕说出错让她没面子,所以很怕打击我,在最喜欢的人面前不敢直视对方(因为对方说话的语句时常很急,扭头缩脚的,因为害怕拒绝而装不下去说错话让对方不乐意),所以我很纠结,我也因为心急,说双方父母这边的帮朋友同事不是自身,而是多余的人互相交流,互相了解答题,做事情很多不像关心。我很迷茫,感觉自己越来越没有主见了,就陷入了死循环。 <eos>
Answer 3: 的爸爸,不到就我们控诉说,所以,可是没人这样的人能年长女孩子高,我们在一起了快三年,她在我知道了,但是我想学识到储蓄咨询师,我和舅舅家距离最远,有点去过不了,要不然他不懂钱财在棵树上诸位。认识的人都长的蛮好的,接受的比自己有钱人多的多,都在省钱,我知道,但是她人言可畏,刚刚还做了一个很大的那个阿姨。我虽然很喜欢和她们住在一起,但由于我也知道母亲并不爱她,她将心里的感觉完全和她们疏远,过了几天,我们就因为这件事情讨论得

## Test

使用 test 语料，从中随机打断，并预测下文，比较原文与预测结果！

随机选 N 个

In [12]:
n_samples = 1000
n_infer = 5
b_random = True
b_shuffle = True

input_file = './data/xinliqa/test.json'
output_file = f'./data/xinliqa/{PARAMETERS}.predict-{latest_checkpointed_iteration}-{n_samples}_{args.seq_length}_{n_infer}'
if b_random:
    output_file += '.random'
if b_shuffle:
    output_file += '.shuffle'
output_file += '.json'

print(f'output_file={output_file}')

total = sum(1 for _ in tqdm(open(input_file)))
print(f'Test 数据总数: {total:,d}')

if n_samples > 0:
    assert total >= n_samples
else:
    n_samples = total

print(f'Test 采样数: {n_samples:,d}')

mask = np.zeros(total, dtype=int)
mask[:n_samples] = 1
if b_random:
    np.random.shuffle(mask)

samples = []
with open(input_file) as fp:
    reader = compress(fp, mask)
    for line in tqdm(reader, 'sample', total=n_samples):
        line = line.strip()
        if not line:
            continue
        data = json.loads(line)
        samples.append(data)

if b_shuffle:
    random.shuffle(samples)

with open(output_file, 'w') as fp:
    for data in tqdm(samples, 'infer'):
        inferred = data['inferred'] = []
        for i in range(n_infer):
            input_ids = question_to_ids(data)
            output_ids = list(infer_tokens_generative(input_ids, model, tokenizer))
            inferred.append(tokenizer.DecodeIds(output_ids))
        print(json.dumps(data, ensure_ascii=False), file=fp)


output_file=./data/xinliqa/117m.predicted-130000-1000_1024_5.random.shuffle.json



Test 数据总数: 4,000
Test 采样数: 1,000
